In [2]:
API_KEY = 'y5Pt4OyhvXKRMNgmQUudrQTVp'
ACESS_TOKEN = '1536862091537178624-BRUEJHXCKJkb9EQP4eMunnmAyP6TDT'
API_KEY_SECRET = 'H2bRlfYoQ9appGSqiQvSONvyitEuttt87fPtYlYhlmYtsRzR5A'
ACESS_TOKEN_SECRET = 'Q6vzz1YsRxOhJ1mDHOXVSt1Id687GuwL1l1yhkRm92SQi'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAI4ZdwEAAAAAr2Z9ihJUyjN2Z5aAsiVFH0zt%2FIM%3DtUFLGD7Fo30mYT5hj2IJwVex27bCSKz2ZAgQDssNKs6HssNBD6'

In [3]:
import sys
!{sys.executable} -m pip install tweepy

You should consider upgrading via the 'c:\Users\eduar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [10]:
import tweepy

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [11]:

data = tweepy.Paginator(client.search_recent_tweets,query='bolsonaro -is:retweet',max_results=100,tweet_fields=['created_at','lang','geo','public_metrics','source'],user_fields=['description'],expansions=['author_id','geo.place_id']).flatten(limit=1000)

In [12]:
for tweet in data:
    print(tweet.data['text'])

@FlanelaVerdeVV @folha É noiiiiis votando em Bolsonaro. Todos tem necessidade de limpar o olho da goiabeira ...Corre e suja sua ✋ . kkkkkkkk
Tereza Cristina surge como a vice ideal para Bolsonaro https://t.co/xFpl2iPrFT
@monark Bolsonaro é uma versão masculina do Monark
A velha mídia fortaleceu o STF para destruir o Bolsonaro. Agora põe um ponto para ser trabalhado caso o Ladrão vença. Se o ladrão vencer, irão trabalhar para enfraquecer o STF. O editorial foi feito a pedido do PT. https://t.co/fXdVj2WrSX
Maior é  Deus na vida do sr Presidente Bolsonaro!!Estamos em Oração por  sua vida e Família Bolsonaro!🙏🏼🙏🏼🙌🏽🙌🏽Deus de todos 🙌🏽🙌🏽 https://t.co/IYmQVwB6K0
@Diego_DS @therockingc @_ale1rbag @neycfreitas @Astronomiaum "Bolsonaro não precisa falar nada para ser narrativa bolsonarista"

O Diabo está em todo lugar mesmo, crente 2.0.
Trabalha mais de graça p sua religião aí, que tá pouco.
@ApoioSilveira Opções:
1 - Vai ter um ataque e exigir que o PGR se explique em 48h
2 - vai incluir o PGR n

In [65]:


# response = client.search_recent_tweets(query='bolsonaro -is:retweet',max_results=100,tweet_fields=['created_at','lang','geo','public_metrics','source'],user_fields=['description'],expansions=['author_id','geo.place_id'])

with open('initial_data','a+', encoding="utf-8") as filehandler:
    for tweet in data:
        filehandler.write('{')
        for k in tweet.data.keys():
            if isinstance(tweet.data[k],dict):
                filehandler.write('(%s' % k)
                for j in tweet.data[k].keys():
                    filehandler.write('[%s,%s];' % (j , tweet.data[k][j]))
                filehandler.write(')')
            else:
                filehandler.write('[%s,%s];' % (k, r''+(tweet.data[k])))
        filehandler.write('}')

UnicodeEncodeError: 'charmap' codec can't encode characters in position 258-259: character maps to <undefined>